<a href="https://colab.research.google.com/github/Deepachandran-S/PATIENT-READMISSION/blob/main/DEPLOYED_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
df.dtypes

encounter_id                float64
patient_nbr                 float64
race                         object
gender                       object
age                          object
admission_type_id           float64
discharge_disposition_id    float64
admission_source_id         float64
time_in_hospital            float64
medical_specialty            object
num_lab_procedures          float64
num_procedures              float64
num_medications             float64
number_outpatient           float64
number_emergency            float64
number_inpatient            float64
diag_1                       object
diag_2                       object
diag_3                       object
number_diagnoses            float64
max_glu_serum                object
A1Cresult                    object
metformin                    object
repaglinide                  object
nateglinide                  object
chlorpropamide               object
glimepiride                  object
acetohexamide               

In [ ]:
'''import warnings
from sklearn.metrics import accuracy_score

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Define the preprocessing function
def preprocess_data(df):
    # Encode missing values as NaN
    df.replace('?', np.nan, inplace=True)

    # Drop columns with more than 50% missing values
    df.drop(['weight', 'payer_code'], axis=1, inplace=True)

    # Fill missing values with a new category 'Missing' for 'medical_specialty'
    df['medical_specialty'].fillna('Missing', inplace=True)

    # Fill missing values for numeric columns with mean and for categorical columns with mode
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=[object]).columns

    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

    # Encode 'readmitted' to binary classes
    df['readmitted'] = df['readmitted'].replace({'<30': 1, '>30': 1, 'NO': 0})

    # Drop patients with expired discharge disposition or discharged to hospice
    df = df[~df['discharge_disposition_id'].isin([11, 19, 20, 21])]

    # Keep only the first encounter for each patient
    df = df.sort_values(by=['patient_nbr', 'encounter_id']).drop_duplicates(subset=['patient_nbr'], keep='first')

    # Encode 'age' as discrete
    age_dict = {'[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35, '[40-50)': 45, '[50-60)': 55,
                '[60-70)': 65, '[70-80)': 75, '[80-90)': 85, '[90-100)': 95}
    df['age'] = df['age'].map(age_dict)

    # Clustering diagnosis codes into 9 categories
    def cluster_diag(diag):
        if diag.startswith('250'):
            return 'Diabetes'
        elif diag.startswith('390') or diag.startswith('459') or diag.startswith('785'):
            return 'Circulatory'
        elif diag.startswith('460') or diag.startswith('519') or diag.startswith('786'):
            return 'Respiratory'
        elif diag.startswith('520') or diag.startswith('579') or diag.startswith('787'):
            return 'Digestive'
        elif diag.startswith('800') or diag.startswith('999'):
            return 'Injury'
        elif diag.startswith('710') or diag.startswith('739'):
            return 'Musculoskeletal'
        elif diag.startswith('580') or diag.startswith('629'):
            return 'Genitourinary'
        elif diag.startswith('140') or diag.startswith('239'):
            return 'Neoplasms'
        else:
            return 'Other'

    df['diag_1'] = df['diag_1'].apply(cluster_diag)
    df['diag_2'] = df['diag_2'].apply(cluster_diag)
    df['diag_3'] = df['diag_3'].apply(cluster_diag)

    # Clustering 'medical_specialty' and other variables
    df['medical_specialty'] = df['medical_specialty'].apply(lambda x: 'Surgery' if 'Surgery' in x else x)

    # Encoding categorical variables
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Feature engineering
    df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
    df['count_medication_change'] = df[['change']].apply(lambda x: x.count(), axis=1)
    df['count_medication_used'] = df[['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
                                      'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
                                      'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
                                      'miglitol', 'troglitazone', 'tolazamide', 'examide',
                                      'citoglipton', 'insulin', 'glyburide-metformin',
                                      'glipizide-metformin', 'glimepiride-pioglitazone',
                                      'metformin-rosiglitazone', 'metformin-pioglitazone']].apply(lambda x: (x != 'No').sum(), axis=1)

    # Drop columns with uniform values
    df.drop(['examide', 'citoglipton'], axis=1, inplace=True)

    # Log transformation for skewed numeric columns
    skewed_cols = ['num_lab_procedures', 'num_procedures', 'num_medications', 'number_diagnoses']
    df[skewed_cols] = df[skewed_cols].apply(lambda x: np.log1p(x))

    # Standardize numeric columns
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    r=df['readmitted']
    df.drop('readmitted',axis=1,inplace=True)
    df=df[['race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_inpatient', 'number_diagnoses', 'A1Cresult', 'metformin',
       'glipizide', 'glyburide', 'pioglitazone', 'insulin', 'change',
       'diabetesMed', 'service_utilization','diag_1','diag_2']]
    return df
# Example usage
df = pd.read_csv('/content/drive/MyDrive/CODIIS/diabetic_data.csv')

preprocessed_data = preprocess_data(df)

# Ensure the data is in the correct shape
#preprocessed_data = preprocessed_data.to_numpy()



# Make prediction
prediction = model.predict(preprocessed_data)
print(f'Prediction: {prediction[0][].round()}')
'''

2202/2202 [==============================] - 4s 2ms/step
Prediction: 0.0


True

In [ ]:
'''import warnings
from sklearn.metrics import accuracy_score

# Suppress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Define the preprocessing function
def preprocess_data(df):
    # Encode missing values as NaN
    df.replace('?', np.nan, inplace=True)

    # Drop columns with more than 50% missing values
    df.drop(['weight', 'payer_code'], axis=1, inplace=True)

    # Fill missing values with a new category 'Missing' for 'medical_specialty'
    df['medical_specialty'].fillna('Missing', inplace=True)

    # Fill missing values for numeric columns with mean and for categorical columns with mode
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=[object]).columns

    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

    # Encode 'readmitted' to binary classes
    df['readmitted'] = df['readmitted'].replace({'<30': 1, '>30': 1, 'NO': 0})

    # Drop patients with expired discharge disposition or discharged to hospice
    df = df[~df['discharge_disposition_id'].isin([11, 19, 20, 21])]

    # Keep only the first encounter for each patient
    df = df.sort_values(by=['patient_nbr', 'encounter_id']).drop_duplicates(subset=['patient_nbr'], keep='first')

    # Encode 'age' as discrete
    age_dict = {'[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35, '[40-50)': 45, '[50-60)': 55,
                '[60-70)': 65, '[70-80)': 75, '[80-90)': 85, '[90-100)': 95}
    df['age'] = df['age'].map(age_dict)

    # Clustering diagnosis codes into 9 categories
    def cluster_diag(diag):
        if diag.startswith('250'):
            return 'Diabetes'
        elif diag.startswith('390') or diag.startswith('459') or diag.startswith('785'):
            return 'Circulatory'
        elif diag.startswith('460') or diag.startswith('519') or diag.startswith('786'):
            return 'Respiratory'
        elif diag.startswith('520') or diag.startswith('579') or diag.startswith('787'):
            return 'Digestive'
        elif diag.startswith('800') or diag.startswith('999'):
            return 'Injury'
        elif diag.startswith('710') or diag.startswith('739'):
            return 'Musculoskeletal'
        elif diag.startswith('580') or diag.startswith('629'):
            return 'Genitourinary'
        elif diag.startswith('140') or diag.startswith('239'):
            return 'Neoplasms'
        else:
            return 'Other'

    df['diag_1'] = df['diag_1'].apply(cluster_diag)
    df['diag_2'] = df['diag_2'].apply(cluster_diag)
    df['diag_3'] = df['diag_3'].apply(cluster_diag)

    # Clustering 'medical_specialty' and other variables
    df['medical_specialty'] = df['medical_specialty'].apply(lambda x: 'Surgery' if 'Surgery' in x else x)

    # Encoding categorical variables
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Feature engineering
    df['service_utilization'] = df['number_outpatient'] + df['number_emergency'] + df['number_inpatient']
    df['count_medication_change'] = df[['change']].apply(lambda x: x.count(), axis=1)
    df['count_medication_used'] = df[['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',
                                      'glimepiride', 'acetohexamide', 'glipizide', 'glyburide',
                                      'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose',
                                      'miglitol', 'troglitazone', 'tolazamide', 'examide',
                                      'citoglipton', 'insulin', 'glyburide-metformin',
                                      'glipizide-metformin', 'glimepiride-pioglitazone',
                                      'metformin-rosiglitazone', 'metformin-pioglitazone']].apply(lambda x: (x != 'No').sum(), axis=1)

    # Drop columns with uniform values
    df.drop(['examide', 'citoglipton'], axis=1, inplace=True)

    # Log transformation for skewed numeric columns
    skewed_cols = ['num_lab_procedures', 'num_procedures', 'num_medications', 'number_diagnoses']
    df[skewed_cols] = df[skewed_cols].apply(lambda x: np.log1p(x))

    # Standardize numeric columns
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    r=df['readmitted']
    df.drop('readmitted',axis=1,inplace=True)
    df=df[['race', 'gender', 'age',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_inpatient', 'number_diagnoses', 'A1Cresult', 'metformin',
       'glipizide', 'glyburide', 'pioglitazone', 'insulin', 'change',
       'diabetesMed', 'service_utilization']]
    return df,r
df = pd.read_csv('/content/drive/MyDrive/CODIIS/diabetic_data.csv')
preprocessed_data = preprocess_data(df)'''

In [ ]:
df2=preprocessed_data
df2[1]

4267      1
5827      0
67608     0
17494     0
2270      0
         ..
99863     0
95282     0
93651     0
101748    1
96147     0
Name: readmitted, Length: 70439, dtype: int64

In [ ]:
import lightgbm as lgb
from sklearn.datasets import make_classification
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score

# Create a sample dataset (replace this with your actual data)
#X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df2[0],df2[1], test_size=0.2, random_state=42)

# Create a LightGBM classifier
lgb_estimator = lgb.LGBMClassifier(class_weight='balanced')


# Fit the RFE model
lgb_estimator.fit(X_train, y_train)

# Transform the training and testing sets based on the selected features


# Fit the LightGBM model on the selected features
#lgb_estimator.fit(X_train_rfe, y_train)

# Predict on the test set
y_pred = lgb_estimator.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred.round())
print(accuracy)

[LightGBM] [Info] Number of positive: 22760, number of negative: 33591
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011840 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 432
[LightGBM] [Info] Number of data points in the train set: 56351, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
0.6142816581487791


In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import pickle

# Load your dataset
# df = pd.read_csv('your_dataset.csv')  # Assuming your dataset is already loaded

  # Assuming 'target' is the column name for the target variable

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df2[0],df2[1], test_size=0.2, random_state=42)

# Define the LightGBM classifier
lgb_clf = lgb.LGBMClassifier(class_weight='balanced')

# Define hyperparameters for tuning
param_grid = {
    'num_leaves': [31, 50],
    'learning_rate': [0.05, 0.1],
    'n_estimators': [100, 200]
}

# Perform hyperparameter tuning with GridSearchCV
grid_search = GridSearchCV(estimator=lgb_clf, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Get the best estimator
best_clf = grid_search.best_estimator_

# Train the model with the best estimator
best_clf.fit(X_train, y_train)

# Make predictions
y_pred = best_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Save the model
with open('lgb_model.pkl', 'wb') as file:
    pickle.dump(best_clf, file)


[LightGBM] [Info] Number of positive: 22760, number of negative: 33591
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012551 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 432
[LightGBM] [Info] Number of data points in the train set: 56351, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Info] Number of positive: 22760, number of negative: 33591
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011812 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 432
[LightGBM] [Info] Number of data points in the train set: 56351, number of used features: 23
[LightGBM] [Info] [b

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred.round())

array([[5093, 3217],
       [2243, 3535]])

In [2]:
import pickle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np

def preprocess_and_save_encoders(df):
    # Encode missing values as NaN
    df.replace('?', np.nan, inplace=True)

    # Columns to preprocess
    columns_to_keep = ['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id',
                       'admission_source_id', 'time_in_hospital', 'medical_specialty',
                       'num_lab_procedures', 'num_procedures', 'num_medications',
                       'number_outpatient', 'number_inpatient', 'number_diagnoses',
                       'A1Cresult', 'metformin', 'glipizide', 'glyburide',
                       'pioglitazone', 'insulin', 'change', 'diabetesMed']
    df = df[columns_to_keep]

    # Fill missing values
    df['medical_specialty'].fillna('Missing', inplace=True)

    # Fill missing values for numeric columns with mean and for categorical columns with mode
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    categorical_cols = df.select_dtypes(include=[object]).columns

    numeric_imputer = SimpleImputer(strategy='mean')
    df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])

    categorical_imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_cols] = categorical_imputer.fit_transform(df[categorical_cols])

    # Encode 'age' as discrete
    age_dict = {'[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35, '[40-50)': 45, '[50-60)': 55,
                '[60-70)': 65, '[70-80)': 75, '[80-90)': 85, '[90-100)': 95}
    df['age'] = df['age'].map(age_dict)

    # Encoding categorical variables
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        label_encoders[col] = le

    # Standardize numeric columns
    scaler = StandardScaler()
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

    # Save the label encoders and the scaler
    with open('label_encoders.pkl', 'wb') as file:
        pickle.dump(label_encoders, file)

    with open('scaler.pkl', 'wb') as file:
        pickle.dump(scaler, file)

    return df

# Example usage
#df = pd.read_csv('/content/drive/MyDrive/CODIIS/diabetic_data.csv')
#df_preprocessed = preprocess_and_save_encoders(df)


<ipython-input-2-8b45d3744b5e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['medical_specialty'].fillna('Missing', inplace=True)
<ipython-input-2-8b45d3744b5e>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = numeric_imputer.fit_transform(df[numeric_cols])
<ipython-input-2-8b45d3744b5e>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 88.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


In [13]:
%%writefile app5.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the saved model, label encoders, and scaler
with open('lgb_model.pkl', 'rb') as file:
    model = pickle.load(file)
with open('label_encoders2.pkl', 'rb') as file:
    label_encoders = pickle.load(file)
#with open('scaler.pkl', 'rb') as file:
    #scaler = pickle.load(file)

# Define the preprocessing function for new input
def preprocess_input(input_df):
    # Encode 'age' as discrete
    age_dict = {'[0-10)': 5, '[10-20)': 15, '[20-30)': 25, '[30-40)': 35, '[40-50)': 45, '[50-60)': 55,
                '[60-70)': 65, '[70-80)': 75, '[80-90)': 85, '[90-100)': 95}
    input_df['age'] = input_df['age'].map(age_dict)
    input_df['A1Cresult']=input_df['A1Cresult'].replace({'None':0, 'Norm':1, '>7':2, '>8':3})
    # Encoding categorical variables
    for col in label_encoders:
      try:
        input_df[col] = label_encoders[col].transform(input_df[col])
      except:
        continue


    # Standardize numeric columns
    numeric_cols = input_df.select_dtypes(include=[np.number]).columns
    input_df[numeric_cols] = StandardScaler().fit_transform(input_df[numeric_cols])

    return input_df

# Define the Streamlit app
st.title('Medical Readmission Prediction')

# Define input fields
race = st.selectbox('Race', ['Caucasian', 'AfricanAmerican', 'Other', 'Asian', 'Hispanic'])
gender = st.selectbox('Gender', ['Male', 'Female'])
age = st.selectbox('Age', ['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)', '[50-60)', '[60-70)', '[70-80)', '[80-90)', '[90-100)'])
admission_type_id = st.selectbox('Admission Type ID', [1, 2, 3, 4, 5, 6, 7, 8])
discharge_disposition_id = st.selectbox('Discharge Disposition ID', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 30])
admission_source_id = st.selectbox('Admission Source ID', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 20, 21, 22, 25, 26])
time_in_hospital = st.number_input('Time in Hospital', min_value=1, max_value=14, value=1)
medical_specialty = st.selectbox('Medical Specialty', ['Cardiology', 'Endocrinology', 'Family/GeneralPractice', 'InternalMedicine', 'Nephrology', 'Neurology', 'ObstetricsandGynecology', 'Oncology', 'Orthopedics', 'Pediatrics', 'Pulmonology', 'Radiology', 'Surgery-General', 'Urology', 'Other'])
num_lab_procedures = st.number_input('Number of Lab Procedures', min_value=0, max_value=132, value=0)
num_procedures = st.number_input('Number of Procedures', min_value=0, max_value=6, value=0)
num_medications = st.number_input('Number of Medications', min_value=1, max_value=81, value=1)
number_outpatient = st.number_input('Number of Outpatient Visits', min_value=0, max_value=42, value=0)
number_inpatient = st.number_input('Number of Inpatient Visits', min_value=0, max_value=21, value=0)
number_diagnoses = st.number_input('Number of Diagnoses', min_value=1, max_value=16, value=1)
A1Cresult = st.selectbox('A1C Result', ['None', 'Norm', '>7', '>8'])
metformin = st.selectbox('Metformin', ['No', 'Steady', 'Up', 'Down'])
glipizide = st.selectbox('Glipizide', ['No', 'Steady', 'Up', 'Down'])
glyburide = st.selectbox('Glyburide', ['No', 'Steady', 'Up', 'Down'])
pioglitazone = st.selectbox('Pioglitazone', ['No', 'Steady', 'Up', 'Down'])
insulin = st.selectbox('Insulin', ['No', 'Steady', 'Up', 'Down'])
change = st.selectbox('Change', ['No', 'Ch'])
diabetesMed = st.selectbox('Diabetes Medication', ['No', 'Yes'])

# Create a dictionary for the input features
input_data = {
    'race': race,
    'gender': gender,
    'age': age,
    'admission_type_id': admission_type_id,
    'discharge_disposition_id': discharge_disposition_id,
    'admission_source_id': admission_source_id,
    'time_in_hospital': time_in_hospital,
    'medical_specialty': medical_specialty,
    'num_lab_procedures': num_lab_procedures,
    'num_procedures': num_procedures,
    'num_medications': num_medications,
    'number_outpatient': number_outpatient,
    'number_inpatient': number_inpatient,
    'number_diagnoses': number_diagnoses,
    'A1Cresult': A1Cresult,
    'metformin': metformin,
    'glipizide': glipizide,
    'glyburide': glyburide,
    'pioglitazone': pioglitazone,
    'insulin': insulin,
    'change': change,
    'diabetesMed': diabetesMed,
    'service_utilization': number_outpatient + number_inpatient
}

# Convert the input data to a DataFrame
input_df = pd.DataFrame([input_data])

# Preprocess the input data
input_df = preprocess_input(input_df)

# Make a prediction
if st.button('Predict'):
    prediction = model.predict(input_df)
    prediction_proba = model.predict_proba(input_df)

    # Display the result
    st.write('Prediction: ', 'Readmitted' if prediction[0].round() == 1 else 'Not Readmitted')
    st.write('PREDICTION PROBABLITY ')
    st.write('NO: ',prediction_proba[0][0])
    st.write('YES: ',prediction_proba[0][1])


Overwriting app5.py


In [15]:
!wget -q -O - https://loca.lt/mytunnelpassword

34.85.185.60

In [17]:
!streamlit run app5.py & npx localtunnel --port 8501 & curl ipv4.icanhazip.com


34.85.185.60



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.85.185.60:8501

npx: installed 22 in 2.886s
your url is: https://yummy-spoons-look.loca.lt
/root/.npm/_npx/3416/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:35533 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/3416/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (events.js:315:20)
    at emitErrorNT (internal/streams/destroy.js:106:8)
    at emitErrorCloseNT (internal/streams/destroy.js:74:3)
    at processTicksAndRejections (internal/process/task_queues.js:80:21)
  Stopping...
